# Build a tile index (`tindex`) for a set of DPS output

In [1]:
from maap.maap import MAAP
#maap = MAAP("api.maap-project.org")
maap = MAAP()

In [2]:
ICESAT2_BOREAL_REPO_PATH = '/projects/code/icesat2_boreal'     
ICESAT2_BOREAL_LIB_PATH = ICESAT2_BOREAL_REPO_PATH + '/lib'

In [3]:
import geopandas as gpd
import pandas as pd
import os

import sys
import s3fs

sys.path.append(ICESAT2_BOREAL_LIB_PATH)

print("Importing packages complete.")

Importing packages complete.


In [4]:
import importlib
import mosaiclib
importlib.reload(mosaiclib)
importlib.reload(maplib_folium)

<module 'maplib_folium' from '/projects/code/icesat2_boreal/lib/maplib_folium.py'>

In [5]:
import mosaiclib
from mosaiclib import *

## Build new tindex and MosiacJSONs

In [9]:
DICT_BUILD_TINDEX = {
    'SET' : 'BOREAL_MAP',
    'USER' : 'lduncanson',
    'ALG_NAME' : 'run_boreal_biomass_map',
    'ALG_VERSION' : 'boreal_agb_2024_v6', 
    'VAR' : 'AGB',
    #'VAR' : 'HT',
    # In my bucket, this is ALWAYS used to identify output
    #'BATCH_NAME' : 'AGB_L30_2020/min_n_1000_funks', #min_n_1000_funks min_n_5000_funks
    #'BATCH_NAME' : 'AGB_H30_2020/outlier_remove_fullboreal',
    'BATCH_NAME' : 'AGB_H30_2020/Version2_SD',
    #'BATCH_NAME' : 'Ht_H30_2020/Version2_SD',
    'YEAR': 2024,
    'DPS_MONTH_LIST' : '07 08 09 10',        
    'DPS_DAY_MIN' : 1 ,
    'TILES_INDEX_PATH': boreal_tile_index_path
}

In [10]:
DICT_BUILD_TINDEX

{'SET': 'BOREAL_MAP',
 'USER': 'lduncanson',
 'ALG_NAME': 'run_boreal_biomass_map',
 'ALG_VERSION': 'boreal_agb_2024_v6',
 'VAR': 'AGB',
 'BATCH_NAME': 'AGB_H30_2020/Version2_SD',
 'YEAR': 2024,
 'DPS_MONTH_LIST': '07 08 09 10',
 'DPS_DAY_MIN': 1,
 'TILES_INDEX_PATH': '/projects/shared-buckets/montesano/databank/boreal_tiles_v004.gpkg'}

In [11]:
%%time
if True:
    DPS_IDENTIFIER = f"{DICT_BUILD_TINDEX['ALG_VERSION']}/{DICT_BUILD_TINDEX['BATCH_NAME']}"

    OUTDIR = f"/projects/my-public-bucket/DPS_tile_lists/{DICT_BUILD_TINDEX['SET']}/{DPS_IDENTIFIER}"
    !mkdir -p $OUTDIR

    # Build tindex
    args = f"--RETURN_DUPS --user {DICT_BUILD_TINDEX['USER']} --dps_identifier {DPS_IDENTIFIER} \
    -alg_name {DICT_BUILD_TINDEX['ALG_NAME']} -t {DICT_BUILD_TINDEX['VAR']} -y 2024 \
    --dps_month_list {DICT_BUILD_TINDEX['DPS_MONTH_LIST']} -d_min {DICT_BUILD_TINDEX['DPS_DAY_MIN']} --outdir {OUTDIR} \
    -boreal_tile_index_path {DICT_BUILD_TINDEX['TILES_INDEX_PATH']}"

    !time /projects/env/above/bin/python /projects/code/icesat2_boreal/lib/build_tindex_master.py $args --WRITE_TINDEX_MATCHES_GDF

NASA MAAP
INFO: Pandarallel will run on 25 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
NASA MAAP

Building a list of tiles:
DPS ID:		boreal_agb_2024_v6/AGB_H30_2020/Version2_SD
Type:		AGB
Year:		['2024']
Month:		['07', '08', '09', '10']
Days:		1-31

Output dir:  /projects/my-public-bucket/DPS_tile_lists/BOREAL_MAP/boreal_agb_2024_v6/AGB_H30_2020/Version2_SD
Rows before NaN removal: 4941
Paths creating null tile_nums: ['s3://maap-ops-workspace/lduncanson/dps_output/run_boreal_biomass_map/boreal_agb_2024_v6/AGB_H30_2020/Version2_SD/2024/07/17/10/33/17/867308/boreal_agb_202407171721237500_4180700.tif', 's3://maap-ops-workspace/lduncanson/dps_output/run_boreal_biomass_map/boreal_agb_2024_v6/AGB_H30_2020/Version2_SD/2024/07/17/10/47/24/011786/boreal_agb_202407171721238347_4199500.tif']
Rows after NaN removal: 4939

No duplicates found.

# of duplicate tiles: 0
Final # of tiles: 4939
Writing duplicates csv: /projects/my-public

### Check the tindex

In [187]:
fn = os.path.join(OUTDIR,f"{DICT_BUILD_TINDEX['VAR']}_tindex_master.csv")
fn

'/projects/my-public-bucket/DPS_tile_lists/BOREAL_MAP/boreal_agb_2024_v6/AGB_H30_2020/Version2_SD/AGB_tindex_master.csv'

In [188]:
tindex_df = pd.read_csv(fn)

In [189]:
tindex_df.shape

(4939, 5)

In [190]:
tindex_df.tail()

,index,s3_path,local_path,file,tile_num
4934,4934,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_2020_202407191721410501_0002232.tif,2232
4935,4935,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_2020_202407191721413363_0001509.tif,1509
4936,4936,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_2020_202407191721416499_0026563.tif,26563
4937,4937,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_2020_202407191721417118_0003690.tif,3690
4938,4938,s3://maap-ops-workspace/lduncanson/dps_output/...,/projects/my-private-bucket/dps_output/run_bor...,boreal_agb_2020_202407191721418631_0002398.tif,2398


In [54]:
tindex_gdf = gpd.read_file(os.path.join(OUTDIR,f"{DICT_BUILD_TINDEX['VAR']}_tindex_master.gpkg"))

In [91]:
#tindex_gdf.explore()